In [ ]:
#匯入模組
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp
from scipy.integrate import odeint
import math

In [ ]:
t,l,m,k,u_b,u_p = smp.symbols('t l m k mu_b mu_p',real=True)
x, y, z = smp.symbols(r'\x \y \z',cls=smp.Function)

x = x(t)
y = y(t)
z = z(t)

x_d = smp.diff(x,t)
y_d = smp.diff(y,t)
z_d = smp.diff(z,t)

x_dd = smp.diff(x_d,t)
y_dd = smp.diff(y_d,t)
z_dd = smp.diff(z_d,t)

\x(t)

In [ ]:
#命名變數
g=9.80665 #(單位:米/秒^2)
l=0.282   #(單位:公尺)
m=0.09  #(單位:公斤)
k=0.3235  #(單位:公尺)
u_b = 1.4043748542 #(單位:安培*公尺^2)
u_p = 4.35534857444  #(單位:安培*公尺^2)
u_0 = 1.25663706212*10**(-6)

In [ ]:
#拉格朗日

#動能
T = 1/2*m*(x_d**2+y_d**2+z_d**2)

#位能
U_g = -m*g*z
U_b = -6.8744*10**(-5)+0.0324*smp.exp(-z/0.05203)*smp.sin(math.pi*(z+4.69227)/0.23256)

#拉格朗日
L = T-U_g+U_b

In [ ]:
constraint = x**2+y**2+z**2-l**2
constraint.simplify()

\x(t)**2 + \y(t)**2 + \z(t)**2 - 0.079524

In [ ]:
#微分、偏微分
LE1 = smp.diff(L,x)-smp.diff(smp.diff(L,x_d),t)
LE1.simplify()
LE2 = smp.diff(L,y)-smp.diff(smp.diff(L,y_d),t)
LE2.simplify()
LE3 = smp.diff(L,z)-smp.diff(smp.diff(L,z_d),t)
LE3.simplify()

#解微分方程
sols = smp.solve([LE1, LE2, LE3, constraint],(x_dd,y_dd,z_dd),simplify=False,rational=False)

In [ ]:
#運動方程1
dz1dt_f = smp.lambdify((x,y,z,x_d,y_d,z_d),sols[x_dd]) #回傳角加速度
dxdt_f = smp.lambdify(x_d,x_d) #回傳角速度=角速度

#運動方程2
dz2dt_f = smp.lambdify((x,y,z,x_d,y_d,z_d),sols[y_dd]) #回傳角加速度
dydt_f = smp.lambdify(y_d,y_d) #回傳角速度=角速度

#運動方程3
dz3dt_f = smp.lambdify((x,y,z,x_d,y_d,z_d),sols[z_dd]) #回傳角加速度
dzdt_f = smp.lambdify(z_d,z_d) #回傳角速度=角速度

#定義會回傳[the、phi、the_d、phi_d]的函式
def dSdt(S,t):
  x,y,z,z1,z2,z3= S
  return [
      dxdt_f(z1),
      dydt_f(z2),
      dzdt_f(z3),
      dz1dt_f(x,y,z,z1,z2,z3),
      dz2dt_f(x,y,z,z1,z2,z3),
      dz3dt_f(x,y,z,z1,z2,z3)]

In [ ]:
#跑數值解
time = 30 #總時長
dt = 0.01 #時間間隔
t = np.linspace(0,time,int(time/dt)) #內插建立時間表

ans = odeint(dSdt,y0=[l,0,0,0,0,0],t=t)




TypeError: Cannot convert expression to float